#### This notebook performs POS tagging and NER Extraction on rap lyric data over 4 decades. It uses a BERT model fine-tuned on Twitter Data. 

In [ ]:
!pip install transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install fastcoref

In [2]:
!unzip /content/merged_all.zip

Archive:  /content/merged_all.zip

  inflating: merged_all.csv          


In [3]:
import pandas as pd

merged_df = pd.read_csv('/kaggle/input/lyrics-mergedall/merged_all.csv')
merged_df.head(2)

/tmp/ipykernel_34/763055692.py:3: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('/kaggle/input/lyrics-mergedall/merged_all.csv')


,Unnamed: 0,artist,song,lyric,next lyric,categories,bio_yearsactivestart,bio_birthdate,bio_summary,bio_yearsactiveend,bio_deathdate,standardized_name,City,State,Country,US_State,Region,release_date,release_datetime,title
0,0,fetty wap,trap queen,rgf productions,remy boyz yahah,['rapper'],2014.0,1991-06-07,"Willie Maxwell II (born June 7, 1991), better ...",NaN,NaN,fetty wap,Paterson,"Paterson, Passaic County, New Jersey, United S...",United States,New Jersey,East,"{'year': 2014, 'month': 3, 'day': 8}",2014-03-08,trap queen
1,0,fetty wap,trap queen,remy boyz yahah,1738 ayy,['rapper'],2014.0,1991-06-07,"Willie Maxwell II (born June 7, 1991), better ...",NaN,NaN,fetty wap,Paterson,"Paterson, Passaic County, New Jersey, United S...",United States,New Jersey,East,"{'year': 2014, 'month': 3, 'day': 8}",2014-03-08,trap queen


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("TweebankNLP/bertweet-tb2_ewt-pos-tagging")
model = AutoModelForTokenClassification.from_pretrained("TweebankNLP/bertweet-tb2_ewt-pos-tagging")

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/537M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
import torch

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Assume merged_df and tokenizer are already defined
lyric_list = merged_df['lyric'].drop_duplicates().tolist()
lyric_list_len = len(lyric_list)

tokens = []
pos_tagged_tokens = []

In [16]:
# for second run
lyric_list =  lyric_list[750394:]
lyric_list_len = len(lyric_list)

In [4]:
import torch
import os

# Enable synchronous error reporting
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
batch_size = 32  # Set your desired batch size

model.to(device)

for i in range(0, len(lyric_list), batch_size):
    batch_lyrics = lyric_list[i:i+batch_size]

    processed_lyrics = tokenizer(
        batch_lyrics,
        return_tensors="pt",
        return_token_type_ids=False,
        return_attention_mask=True,
        truncation=True,
        padding=True,
        max_length=tokenizer.model_max_length
    ).to(device)

    with torch.no_grad():
        output = model(**processed_lyrics)

    for j in range(len(batch_lyrics)):
        token_ids = processed_lyrics["input_ids"][j].squeeze().tolist()
        processed_tokens = tokenizer.convert_ids_to_tokens(token_ids)
        predicted_tags = torch.argmax(output.logits[j], dim=1).squeeze().tolist()
        pos_tags = [model.config.id2label[tag] for tag in predicted_tags]

        tokens.append(processed_tokens)
        pos_tagged_tokens.append(pos_tags)

    print(f"Processed {min(i + batch_size, len(lyric_list))}/{len(lyric_list)} lyrics..")


In [22]:

pos_tags_df = pd.DataFrame({
    'lyric': lyric_list,
    'tokens': tokens,
    'pos_tags': pos_tagged_tokens
})


pos_tags_df.to_csv("pos_tagged_lyrics_lastpart.csv")

In [24]:
#from kaggle import files
#files.download('pos_tagged_lyrics.csv')
!mv /kaggle/working/pos_tagged_lyrics_lastpart.zip /kaggle/working/pos_tagged_lyrics_lastpart.zip

mv: cannot stat '/kaggle/working/pos_tagged_lyrics_lastpart.zip': No such file or directory


In [25]:
!zip pos_tagged_lyrics_lastpart.zip pos_tagged_lyrics_lastpart.csv

  adding: pos_tagged_lyrics_lastpart.csv (deflated 83%)


In [6]:
def resolve_lyrics(preds, print_out=False):
    resolved_lyrics = []

    for pred in preds:
        texts = pred.text
        tuples = pred.get_clusters(as_strings=False)

        if not tuples:
            resolved_lyrics.append(texts)
            continue

        if len(tuples) == 1:
            resolved_lyrics.append(texts)
            continue 
        longest_word = find_longest_cluster(tuples)
        offset = 0
        for i, (start, end) in enumerate(tuples[0]):
            if i == 0:
                start_idx = start
                end_idx = end
            else:
                original_text = texts[start:end]
                replacement_text = texts[start_idx:end_idx]

                texts = texts[:start + offset] + replacement_text + texts[end + offset:]
                offset += len(replacement_text) - (end - start)

                if print_out:
                    print('*' * 50)
                    print('Replacing', original_text, 'with', replacement_text)
                    print('Replaced text:', texts)
                    print('*' * 50, '\n')

        resolved_lyrics.append(texts)

    return resolved_lyrics


In [7]:
batch_size = 256  # Set the batch size as desired

tmp_df = merged_df.drop_duplicates(subset=['lyric'])
length = len(tmp_df)

In [17]:
unique_lyrics = tmp_df['lyric'].values[:length // 2]
#unique_lyrics = tmp_df['lyric'].values[length // 2:]


# Split the unique lyrics into batches
unique_lyrics_batches = [unique_lyrics[i:i+batch_size] for i in range(0, len(unique_lyrics), batch_size)]
unique_lyrics_batches_len =len(unique_lyrics_batches)
# Get unique lyrics in batches

In [ ]:
#!pip install fastcoref

from fastcoref import FCoref

#model = FCoref()
model = FCoref(device='cuda:0')

In [ ]:
#%%capture 
resolved_lyrics = []
lyrics = []

# Iterate over unique lyrics batches
for i,batch in enumerate(unique_lyrics_batches):
    if (i+1) % 100 == 0:
        print(f"Processing batch {i}/{unique_lyrics_batches_len}..")
    preds = model.predict(texts=batch.tolist())  # Passing the batch of lyrics to the model
    resolved_batch = resolve_lyrics(preds,print_out=False)  # Assuming resolve_lyrics() processes a batch of lyrics

    # Add the resolved lyrics and original lyrics to their respective lists

    resolved_lyrics.extend(resolved_batch)
    lyrics.extend(batch)
        # Do something with resolved_lyrics
    # Remove break if you want to iterate over all songs in batches

In [ ]:
processed_resolved_lyrics = resolved_lyrics.copy()
processed_lyrics = lyrics.copy()

In [ ]:
import pandas as pd

# Assuming `lyrics` and `resolved_lyrics` are lists with single elements
# lyrics = ["original lyric 1"]
# resolved_lyrics = resolve_lyrics(preds)

# Create the DataFrame
coref_df = pd.DataFrame({
    'lyric': lyrics,
    'resolved_lyric': resolved_lyrics
})

coref_df.to_csv('coref_df.csv')

In [ ]:
coref_df.to_csv('coref_df.csv')

In [6]:
coref1 = pd.read_excel('/kaggle/input/coreflyricsfix/coref.xlsx')
coref2 = pd.read_csv('/kaggle/input/coreflyricsfix/coref_df_second_part.csv')
# Concatenate the two DataFrames


In [9]:
coref_df = pd.concat([coref1,coref2], ignore_index=True)
coref_df.head()

,lyric,resolved_lyric
0,rgf productions,rgf productions
1,remy boyz yahah,remy boyz yahah
2,1738 ayy,1738 ayy
3,im like hey whats up hello,im like hey whats up hello
4,seen yo pretty ass soon as you came in the door,seen yo pretty ass soon as you came in the door


NER Tagging all lyrics

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

ner_model = AutoModelForTokenClassification.from_pretrained('/kaggle/input/nermodelconfig/results')
ner_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/nermodelconfig/results')

In [12]:
ner_model.config

BertConfig {
  "_name_or_path": "/kaggle/input/nermodelconfig/results",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-corporation",
    "2": "I-corporation",
    "3": "B-creative-work",
    "4": "I-creative-work",
    "5": "B-group",
    "6": "I-group",
    "7": "B-location",
    "8": "I-location",
    "9": "B-person",
    "10": "I-person",
    "11": "B-product",
    "12": "I-product"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-corporation": 1,
    "B-creative-work": 3,
    "B-group": 5,
    "B-location": 7,
    "B-person": 9,
    "B-product": 11,
    "I-corporation": 2,
    "I-creative-work": 4,
    "I-group": 6,
    "I-location": 8,
    "I-person": 10,
    "I-product": 12,
    "O": 0
  },
  "layer_norm_eps": 1e-

In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
lyric_list = coref_df['lyric'].drop_duplicates().tolist()
lyric_list_len = len(lyric_list)

ner_tokens = []
ner_tagged_tokens = []

In [ ]:
ner_model.to(device)

batch_size = 32  # Set your desired batch size

for i in range(0, len(lyric_list), batch_size):
  try:    
        batch_lyrics = lyric_list[i:i+batch_size]
        processed_lyrics = tokenizer(
            batch_lyrics,
            return_tensors="pt",
            return_token_type_ids=False,
            return_attention_mask=True,
            truncation=True,
            padding=True,
            max_length=tokenizer.model_max_length
        ).to(device)
        
        with torch.no_grad():
            output = ner_model(**processed_lyric)
        
        for j in range(len(batch_lyrics)):
            token_ids = processed_lyrics["input_ids"][j].squeeze().tolist()
            processed_tokens = ner_tokenizer.convert_ids_to_tokens(token_ids)
            predicted_tags = torch.argmax(output.logits[j], dim=1).squeeze().tolist()
            ner_tags = [ner_model.config.id2label[tag] for tag in predicted_tags]
             
            ner_tokens.append(processed_tokens)
            ner_tagged_tokens.append(pos_tags)

        if (i+1) % 10000 == 0:
            print(f"Processed {i+1}/{lyric_list_len} lyrics..")
            break

        # Optional: if you need the results, you can store them
        # results.append(tokens_with_tags)
  except Exception as e:
    print(f"Error processing lyric {i+1}: {e}")
    tokens.append('N/A')
    pos_tagged_tokens.append('N/A')
    break
    continue
        

In [16]:
import torch

ner_model.to(device)

batch_size = 32  # Set your desired batch size
ner_tokens = []
ner_tagged_tokens = []
num_batches = (len(lyric_list) + batch_size - 1) // batch_size  # Calculate total number of batches


In [44]:
ner_tokens = []
ner_tagged_tokens = []

for i in range(0, len(lyric_list), batch_size):
    try:
        batch_lyrics = lyric_list[i:i+batch_size]
        processed_lyrics = ner_tokenizer(
            batch_lyrics,
            return_tensors="pt",
            return_token_type_ids=False,
            return_attention_mask=True,
            truncation=True,
            padding=True,
            max_length=ner_tokenizer.model_max_length
        ).to(device)
        
        with torch.no_grad():
            output = ner_model(**processed_lyrics)
        
        for j in range(len(batch_lyrics)):
            token_ids = processed_lyrics["input_ids"][j].squeeze().tolist()
            processed_tokens = ner_tokenizer.convert_ids_to_tokens(token_ids)
            predicted_tags = torch.argmax(output.logits[j], dim=1).squeeze().tolist()
            ner_tags = [ner_model.config.id2label[tag] for tag in predicted_tags]
             
            ner_tokens.append(processed_tokens)
            ner_tagged_tokens.append(ner_tags)
        
        current_batch = (i // batch_size) + 1
        if current_batch % 1000 == 0:
            print(f"Processed {current_batch}/{num_batches} batches..")
        
    except Exception as e:
        print(f"Error processing lyric {i+1}: {e}")
        ner_tokens.append('N/A')
        ner_tagged_tokens.append('N/A')

Error processing lyric 30977: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Processed 1000/32168 batches..
Processed 2000/32168 batches..
Processed 3000/32168 batches..
Processed 4000/32168 batches..
Processed 5000/32168 batches..
Processed 6000/32168 batches..
Processed 7000/32168 batches..
Processed 8000/32168 batches..
Processed 9000/32168 batches..
Processed 10000/32168 batches..
Processed 11000/32168 batches..
Processed 12000/32168 batches..
Processed 13000/32168 batches..
Processed 14000/32168 batches..
Processed 15000/32168 batches..
Processed 16000/32168 batches..
Processed 17000/32168 batches..
Processed 18000/32168 batches..
Processed 19000/32168 batches..
Processed 20000/32168 batches..
Processed 21000/32168 batches..
Processed 22000/32168 batches..
Processed 23000/32168 batches..
Processed 24000/32168 batches..
Processed 25000/32168 batches..
Processed 26000/32168 batches..
Processed 27000/32168 batches..
Processed 28000/32168 batches

In [110]:
# due to error a batch was not added, so will just remove it from consideration
start = 30976
end =  31007
lyric_list_fix = lyric_list[:start] + lyric_list[end:]

In [125]:
ner_tags_df = pd.DataFrame({
    'lyric': lyric_list_fix,
    'tokens': ner_tokens,
    'ner_tags': ner_tagged_tokens
})

ner_tags_df.to_csv("ner_tagged_lyrics.csv")

In [ ]:
!zip ner_tagged_lyrics.zip /kaggle/working/ner_tagged_lyrics.csv